In [ ]:
# coding=utf-8
# Copyright 2022 The HuggingFace Inc. team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, pipeline

In [ ]:
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer, set_seed
from trl.core import LengthSampler

######################################################################<br>
This is a fully working simple example to use trl with accelerate.<br>
<br>
This example fine-tunes a GPT2 model on the IMDB dataset using PPO<br>
(proximal policy optimization).<br>
in any of the following settings (with the same script):<br>
  - single CPU or single GPU<br>
  - fp16 (mixed-precision) or fp32 (normal precision)<br>
<br>
To run it in each of these various modes, first initialize the accelerate<br>
configuration with `accelerate config`<br>
<br>
######################################################################

######################################################################<br>
NOTE for to train with a 8-bit model a more recent version of<br>
transformers is required, full dependecies for this example:<br>
pip install  bitsandbytes datasets accelerate loralib<br>
pip install  git+https://github.com/huggingface/transformers.git@main<br>
pip install peft<br>
######################################################################

We first define the configuration of the experiment, defining the model, the dataset,<br>
the training parameters, and the PPO parameters.<br>
Check the default arguments in the `PPOConfig` class for more details.<br>
If you want to log with tensorboard, add the kwarg<br>
`project_kwargs={"logging_dir": PATH_TO_LOGS}` to the PPOConfig.

Define and parse arguments.

In [ ]:
@dataclass
class ScriptArguments:
    """
    The name of the Casual LM model we wish to fine with PPO
    """

    # NOTE: gpt2 models use Conv1D instead of Linear layers which are not yet supported in 8 bit mode
    # models like gpt-neo* models are more suitable.
    model_name: Optional[str] = field(
        default="edbeeching/gpt-neo-125M-imdb-lora-adapter-merged", metadata={"help": "the model name"}
    )
    log_with: Optional[str] = field(default=None, metadata={"help": "use 'wandb' to log with wandb"})
    learning_rate: Optional[float] = field(default=1.41e-5, metadata={"help": "the learning rate"})
    mini_batch_size: Optional[int] = field(default=16, metadata={"help": "the PPO minibatch size"})
    batch_size: Optional[int] = field(default=256, metadata={"help": "the batch size"})
    gradient_accumulation_steps: Optional[int] = field(
        default=1, metadata={"help": "the number of gradient accumulation steps"}
    )

In [ ]:
parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses()[0]

In [ ]:
config = PPOConfig(
    model_name=script_args.model_name,
    learning_rate=script_args.learning_rate,
    log_with=script_args.log_with,
    mini_batch_size=script_args.mini_batch_size,
    batch_size=script_args.batch_size,
    gradient_accumulation_steps=script_args.gradient_accumulation_steps,
)

We then define the arguments to pass to the sentiment analysis pipeline.<br>
We set `return_all_scores` to True to get the sentiment score for each token.

In [ ]:
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": config.mini_batch_size}

Below is an example function to build the dataset. In our case, we use the IMDB dataset<br>
from the `datasets` library. One should customize this function to train the model on<br>
its own dataset.

In [ ]:
def build_dataset(config, dataset_name="imdb", input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.
    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.
    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(dataset_name, split="train")
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)
    input_size = LengthSampler(input_min_text_length, input_max_text_length)
    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["review"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample
    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds

We retrieve the dataloader by calling the `build_dataset` function.

In [ ]:
dataset = build_dataset(config)

In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

set seed before initializing value head for deterministic eval

In [ ]:
set_seed(config.seed)

Now let's build the model, the reference model, and the tokenizer.

In [ ]:
pretrained_model = AutoModelForCausalLM.from_pretrained(config.model_name, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.model_name)


### Apply LoRA<br>
Here comes the magic with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`.<br>


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
target_modules = None
if "gpt-neox" in script_args.model_name:
    target_modules = ["query_key_value", "xxx"]  # workaround to use 8bit training on this model

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=target_modules,  # handled automatically by peft
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
pretrained_model = prepare_model_for_int8_training(pretrained_model, output_embedding_layer_name="embed_out")

hacky workaround due to issues with "EleutherAI/gpt-neox-20b"

In [ ]:
if "gpt-neox" in script_args.model_name:
    for name, param in pretrained_model.named_parameters():
        # freeze base model's layers
        param.requires_grad = False
        if getattr(pretrained_model, "is_loaded_in_8bit", False):
            # cast layer norm in fp32 for stability for 8bit models
            if param.ndim == 1 and "layer_norm" in name:
                param.data = param.data.to(torch.float16)

In [ ]:
pretrained_model = get_peft_model(pretrained_model, lora_config)

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(pretrained_model)

In [ ]:
model.gradient_checkpointing_disable = model.pretrained_model.gradient_checkpointing_disable
model.gradient_checkpointing_enable = model.pretrained_model.gradient_checkpointing_enable

In [ ]:
print_trainable_parameters(model)

GPT-2 tokenizer has a pad token, but it is not eos_token by default. We need to set it to eos_token.<br>
only for this model.

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

We then build the PPOTrainer, passing the model, the reference model, the tokenizer

In [ ]:
ppo_trainer = PPOTrainer(
    config, model, ref_model=None, tokenizer=tokenizer, dataset=dataset, data_collator=collator, optimizer=optimizer
)

We then build the sentiment analysis pipeline, passing the model name and the<br>
sentiment analysis pipeline arguments. Let's also make sure to set the device<br>
to the same device as the PPOTrainer.

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

We then define the arguments to pass to the `generate` function. These arguments<br>
are passed to the `generate` function of the PPOTrainer, which is a wrapper around<br>
the `generate` function of the trained model.

In [ ]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "eos_token_id": -1,
}
output_min_length = 4
output_max_length = 16
output_length_sampler = LengthSampler(output_min_length, output_max_length)

In [ ]:
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    model.gradient_checkpointing_disable()
    model.pretrained_model.config.use_cache = True
    # Get response from Causal LM
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    # Compute sentiment score
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    # Run PPO step
    model.gradient_checkpointing_enable()
    model.pretrained_model.config.use_cache = False
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

In [ ]:
model.push_to_hub(f"{script_args.model_name}-ppo-sentiment")